Imports

In [32]:
from tensorflow import keras
import tensorflow as tf
from keras import layers
from keras.datasets.cifar import load_batch
from keras import backend
import numpy as np
import os


#Dataset Load

In [33]:
path = "datasets"
num_train_samples = 50000
x_train = np.empty((num_train_samples, 3, 32, 32), dtype="uint8")
y_train = np.empty((num_train_samples,), dtype="uint8")
for i in range(1, 6):
    fpath = os.path.join(path, "data_batch_" + str(i))
    (x_train[(i - 1) * 10000 : i * 10000, :, :, :],y_train[(i - 1) * 10000 : i * 10000],) = load_batch(fpath)
fpath = os.path.join(path, "test_batch")
x_test, y_test = load_batch(fpath)
y_train = np.reshape(y_train, (len(y_train), 1))
y_test = np.reshape(y_test, (len(y_test), 1))
if backend.image_data_format() == "channels_last":
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)
x_test = x_test.astype(x_train.dtype)
y_test = y_test.astype(y_train.dtype)
print(x_train.shape,x_test.shape)
# (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


(50000, 32, 32, 3) (10000, 32, 32, 3)


In [34]:
num_classes = 10
img_rows, img_cols = 32, 32

Normalise

In [35]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [36]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Model definition

In [37]:
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(img_rows, img_cols, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
782/782 [==============================] - 21s 25ms/step - loss: 1.5677 - accuracy: 0.4292
Epoch 2/5
782/782 [==============================] - 21s 26ms/step - loss: 1.2068 - accuracy: 0.5736
Epoch 3/5
782/782 [==============================] - 21s 26ms/step - loss: 1.0427 - accuracy: 0.6305
Epoch 4/5
782/782 [==============================] - 21s 26ms/step - loss: 0.9463 - accuracy: 0.6688
Epoch 5/5
782/782 [==============================] - 21s 26ms/step - loss: 0.8729 - accuracy: 0.6938


# evaluate the model

In [39]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

313/313 - 1s - loss: 0.8953 - accuracy: 0.6872 - 1s/epoch - 5ms/step
Test accuracy: 0.6872000098228455


Saving the Keras model

In [43]:
model.save('saved_model/Classification')




INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmp_4eztu9f\assets


INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmp_4eztu9f\assets


converting and saving the Lite version of the Model  

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('saved_model/LiteModel.tflite', 'wb') as f:
    f.write(tflite_model)